In [12]:
import requests
import pathlib
import json

import socket
import socks

import datetime
import time

import urllib.parse
from urllib.parse import urlparse, parse_qs, unquote

import sqlite3

In [13]:
# set socks5 proxy
socks.set_default_proxy(socks.SOCKS5, '127.0.0.1', 13659)
socket.socket = socks.socksocket


In [18]:
# read report schema
with open('./data/report_schema.json', 'r') as f:
    report_schema = json.load(f)
report_schema
print(report_schema['income_statement'])

['TaxEffectOfUnusualItems', 'TaxRateForCalcs', 'NormalizedEBITDA', 'NormalizedDilutedEPS', 'NormalizedBasicEPS', 'TotalUnusualItems', 'TotalUnusualItemsExcludingGoodwill', 'NetIncomeFromContinuingOperationNetMinorityInterest', 'ReconciledDepreciation', 'ReconciledCostOfRevenue', 'EBITDA', 'EBIT', 'NetInterestIncome', 'InterestExpense', 'InterestIncome', 'ContinuingAndDiscontinuedDilutedEPS', 'ContinuingAndDiscontinuedBasicEPS', 'NormalizedIncome', 'NetIncomeFromContinuingAndDiscontinuedOperation', 'TotalExpenses', 'RentExpenseSupplemental', 'ReportedNormalizedDilutedEPS', 'ReportedNormalizedBasicEPS', 'TotalOperatingIncomeAsReported', 'DividendPerShare', 'DilutedAverageShares', 'BasicAverageShares', 'DilutedEPS', 'DilutedEPSOtherGainsLosses', 'TaxLossCarryforwardDilutedEPS', 'DilutedAccountingChange', 'DilutedExtraordinary', 'DilutedDiscontinuousOperations', 'DilutedContinuousOperations', 'BasicEPS', 'BasicEPSOtherGainsLosses', 'TaxLossCarryforwardBasicEPS', 'BasicAccountingChange', 'B

In [57]:
# loop the is_yf_12m.json

def gen_field_id_dict(file_name):
    
    with open(file_name, 'r') as fd:
        is_yf = json.load(fd)

    field_id_dict = {}

    result = is_yf['timeseries']['result']
    for i, r in enumerate(result):
        m = r['meta']
        f = m['type'][0]

        if f not in r.keys():
            # print(f'--> the result[{i}] has not the key: [{f}].')
            continue

        for j, t in enumerate(r['timestamp']):
            d = r[f][j]
            if d:
                f = f[8:] if f.startswith('trailing') else f[6:] if f.startswith('annual') else f[9:]
                if f in field_id_dict.keys():
                    d1 = field_id_dict[f]
                    if d1 != d['dataId']:
                        print(f'{f} with different dataId {d1}')

                field_id_dict[f] = d['dataId']

                break

    return field_id_dict


def gen_report_schema(report_schema, report_file):
    
    field_id_dict = gen_field_id_dict(report_file)
    print('len of field id', len(field_id_dict.keys()))
    
    yf_schema = []
    len_with_id = 0
    for field in report_schema:
        
        yf_f_schema = []
        yf_f_schema.append(field)
        
        if field not in field_id_dict.keys():
    #         print(f'---> field: {field} without field id')
            yf_f_schema.append(0)
        else:
    #         print(field_id)
            field_id = field_id_dict[field]
            yf_f_schema.append(field_id)
            len_with_id = len_with_id + 1
            
        yf_schema.append(yf_f_schema)

    return yf_schema, len_with_id



In [63]:
with open('./data/report_schema.json', 'r') as f:
    report_schema = json.load(f)
    
is_schema = report_schema['cash_flow']
is_schema.reverse()

report_schema, len_with_id = gen_report_schema(is_schema, './data/cf_yf_12m.json')
print('len_with_id:', len_with_id)
report_schema

len of field id 73
len_with_id: 73


[['ReceiptsfromCustomers', 0],
 ['ReceiptsfromGovernmentGrants', 0],
 ['OtherCashReceiptsfromOperatingActivities', 0],
 ['ClassesofCashReceiptsfromOperatingActivities', 0],
 ['PaymentstoSuppliersforGoodsandServices', 0],
 ['PaymentsonBehalfofEmployees', 0],
 ['OtherCashPaymentsfromOperatingActivities', 0],
 ['ClassesofCashPayments', 0],
 ['DividendsPaidDirect', 0],
 ['DividendsReceivedDirect', 0],
 ['InterestPaidDirect', 0],
 ['InterestReceivedDirect', 0],
 ['TaxesRefundPaidDirect', 0],
 ['CashFlowsfromusedinOperatingActivitiesDirect', 0],
 ['NetIncomeFromContinuingOperations', 26080],
 ['GainLossOnSaleOfBusiness', 26159],
 ['GainLossOnSaleOfPPE', 26160],
 ['NetForeignCurrencyExchangeGainLoss', 26078],
 ['GainLossOnInvestmentSecurities', 26158],
 ['EarningsLossesFromEquityInvestments', 26155],
 ['PensionAndEmployeeBenefitExpense', 0],
 ['OperatingGainsLosses', 26163],
 ['Depreciation', 26059],
 ['AmortizationOfIntangibles', 0],
 ['AmortizationCashFlow', 0],
 ['DepreciationAndAmortizati